In [1]:
import msmbuilder.clustering
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import cv2
import os
import random
import pylab as pl
import numpy as np
import cv2 as cv
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from active_semi_clustering.semi_supervised.pairwise_constraints import PCKMeans,COPKMeans, MPCKMeansMF, MPCKMeans
from active_semi_clustering.active.pairwise_constraints import ExampleOracle, ExploreConsolidate, MinMax
import active_semi_clustering.active.pairwise_constraints.random
import pickle
import shap
import imagehash
from PIL import Image
from operator import itemgetter

In [2]:
builder = tfds.builder('voc/2007')
builder.download_and_prepare()
datasets = builder.as_dataset()
train_data, val_data, test_data = datasets['train'], datasets['validation'], datasets['test']


In [3]:
des_list=[]
images = train_data.map(lambda x: x["image"])

In [4]:
orb= cv.ORB_create()
for image_pat in images:

    kp=orb.detect(image_pat.numpy(),None)
    keypoints,descriptor= orb.compute(image_pat.numpy(), kp)
    des_list.append(descriptor)

In [5]:
print(len(des_list))

2501


In [6]:
#print(keypoints)

In [7]:
des_list[0]

array([[173,  68, 213, ..., 251,  66, 160],
       [ 43, 160, 176, ...,  12, 222, 198],
       [202, 189, 255, ..., 244, 199, 127],
       ...,
       [120,  15, 114, ..., 171, 124, 130],
       [255,  87, 190, ..., 247, 102, 119],
       [227,  38,  81, ..., 208, 198, 100]], dtype=uint8)

In [8]:
descriptors=np.vstack(des_list)

In [9]:
descriptors.shape

(1227245, 32)

In [10]:
print(descriptors[0])

[173  68 213 111  13 110 103  24 230 178 231  88 211 113  25 251  73 212
 122 106 105 244 217  10 199 251  69  20 218 251  66 160]


In [11]:
#descriptors_float=descriptors.astype(float)

In [12]:
#!pip install scikit-learn-extra

In [13]:
from scipy.spatial.distance import hamming

In [14]:
out_arr = np.unpackbits(descriptors, axis = 1)

In [15]:
out_arr[2]

array([1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1], dtype=uint8)

In [16]:
from scipy.spatial.distance import pdist

In [17]:
#pip install scikit-learn-extra

In [18]:
from sklearn_extra.cluster import KMedoids
from sklearn.metrics import make_scorer

In [19]:
#dt=DistanceMetric.get_metric('pyfunc',func=ham_dis)

In [20]:
#score = make_scorer(ham_dis)


In [21]:
from scipy.spatial import distance
import time


class KMajority:
    def __init__(self,n_clusters=8,max_iter=100,):
        
        self.n_cluster=n_clusters
        self.max_iter=max_iter
        
    def fit(self,X,y=None,centroid=None):
        X=np.array(X, dtype=bool)
        #random centriods
        if centroid is None:
            centroids=np.random.randint(2, size=(self.n_cluster,X.shape[1]))
            self.centroids=np.array(centroids, dtype=bool)
        else:
            self.centroids=centroid
        centroids=None
            
        for i in range(self.max_iter):
            #distance from each point in X and the centroids so ouput is shape (number_samples, number_clusters)
            number_samples =X.shape[0]
            number_features=X.shape[1]
            
            dist=distance.cdist(X,self.centroids,'hamming')
            #shape (number_samples,)
            cluster_assigned=np.argmin(dist,axis=1)
            
            
            #we calculate one hot encodings to select the rows we want to sum over using matrix multiplication
            #test=np.zeros((X.shape[0],self.n_cluster))
            #test[np.arange(X.shape[0]),cluster_assigned]=1
            #out=test.T@X
            
            row_sum = np.array([X[cluster_assigned==i].sum(axis=0) for i in range(self.n_cluster)])
            row_count = np.array([(cluster_assigned==i).sum(axis=0) for i in range(self.n_cluster)])

            #print(row_sum)
            #print(row_count)
            new_centroids=row_sum>(row_count/2).reshape(-1,1)
            #remove empty clusering init randomly again nets bad results but setting it to actual points helps
            nullcheck=row_count==0
            if(np.any(nullcheck)):
                for k in np.argwhere(nullcheck):
                    #does not work to bad performance in comparision to already fitted clusters
                    #zw=np.random.randint(2, size=(X.shape[1]))
                    
                    index = np.random.choice(X.shape[0], 1, replace=False) 
                    new_centroids[k[0]]=X[index]
                    
            #print(centroids[0])
            #print(new_centroids[0])
            print(i)
            if(np.array_equal(self.centroids,new_centroids)):
                self.labels=cluster_assigned
                self.centroids=new_centroids
                return self
            
            self.labels=cluster_assigned
            self.centroids=new_centroids
            
        return self
            
    def predict(self,X):
        X=np.array(X, dtype=bool)
            #distance from each point in X and the centroids so ouput is shape (number_samples, number_clusters)
        number_samples =X.shape[0]
        number_features=X.shape[1]
        dist=distance.cdist(X,self.centroids,'hamming')
        cluster_assigned=np.argmin(dist,axis=1)
        return cluster_assigned

            


In [22]:
kmajority = KMajority(n_clusters=1000,max_iter=25)
import pickle

In [23]:
index = np.random.choice(out_arr.shape[0], 20000, replace=False) 
kmajority.max_iter=50
start = time.time()
kmajority.fit(out_arr[index])
end = time.time()
print(end-start)
with open('dumped_centroids1.pkl', 'wb') as fid:
    pickle.dump(kmajority.centroids, fid)   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
167.45888090133667


In [24]:
index = np.random.choice(out_arr.shape[0], 200000, replace=False)
kmajority.max_iter=75
start = time.time()
kmajority.fit(out_arr[index],centroid=kmajority.centroids)
end = time.time()
print(end-start)
with open('dumped_centroids2.pkl', 'wb') as fid:
    pickle.dump(kmajority.centroids, fid)   

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
1866.8678879737854


In [25]:
kmajority.max_iter=25

start = time.time()
kmajority.fit(out_arr,centroid=kmajority.centroids)
end = time.time()
print(end-start)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


KeyboardInterrupt: 

In [27]:

with open('dumped_centroids.pkl', 'wb') as fid:
    pickle.dump(kmajority.centroids, fid)   
    
with open('dumped_kmajority.pkl', 'wb') as fid2:
    pickle.dump(kmajority, fid2)   

In [ ]:
#kmajority.fit(zw,centroid=kmajority.centroids)

In [191]:
kmajority.predict(out_arr[index])

array([198, 882, 617, ..., 655, 364, 559], dtype=int64)

In [32]:
!pip install mdtraj

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for mdtraj: filename=mdtraj-1.9.7-cp37-cp37m-win_amd64.whl size=1441507 sha256=503b6d61dab3c27ad8b5b6eea381154e6691fdd475253ead9f7f1e4b51838704
  Stored in directory: c:\users\root\appdata\local\pip\cache\wheels\6f\84\9d\6854e5264a4423209de26a6e4b34d13750289c72ba0439bbfd
Successfully built mdtraj


In [ ]:
!pip install msmbuilder

In [37]:
!pip install tables

ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml
unclosed socket <zmq.Socket(zmq.PUSH) at 0x253331c9f98>
unclosed file <_io.BufferedWriter name=4>
unclosed file <_io.BufferedReader name=5>
unclosed file <_io.BufferedReader name=6>


In [40]:
!pip install pyyaml


unclosed socket <zmq.Socket(zmq.PUSH) at 0x253331df748>
unclosed file <_io.BufferedWriter name=4>
unclosed file <_io.BufferedReader name=5>
unclosed file <_io.BufferedReader name=6>


ImportError: cannot import name 'cluster' from 'msmbuilder' (C:\Users\root\anaconda3\envs\ATiML_Project\lib\site-packages\msmbuilder\__init__.py)

NameError: name 'msmbuilder' is not defined